### Coding 3: Final Assignment - Bechdel Test Investigation
#### Peter Endes-Nagy and Nadine Levin

## Introduction

Lorem ipsum

*Ideas*
1. Scrape IMDb site for LGBT films of last 10 years to compare to popular films of last 10 years
2. Investigate if LGBT films score higher on the Bechdel test than popular films
3. Investigate if women direct films that score higher on the Bechdel test


*Resources* 
1. https://data.world/sharon/bechdel-test
2. https://bechdeltest.com/
3. https://www.kaggle.com/rounakbanik/the-movies-dataset
4. https://www.themoviedb.org/documentation/api

In [1]:
import pandas as pd 
import numpy as np 
import re
import requests
from bs4 import BeautifulSoup
import time
import json
import string
import html
#!pip install gender-guesser
import gender_guesser.detector as gen 

### Base dataset

Our base is the full Bechdel dataset as the Bechdel score is one of our key variables. Called via the Bechdel API only once, the curators of the dataset explicitly ask us not to call it too often.  

In later steps, we:  
1) Scrape the tags associated with these movies from the IMDb website to figure out the LGBTQ+ films  
2) Get additional data on the movies via OMDb API

In [ ]:
bechdel_films = pd.read_json('http://bechdeltest.com/api/v1/getAllMovies') 

It's HTML encoded, so let's clean the Movie Titles:

In [ ]:
bechdel_films['title'] = bechdel_films['title'].apply(lambda x: html.unescape(x) )

Save it into a .csv, so we won't need to abuse the Bechdel API server: 

In [ ]:
#bechdel_films.to_csv( 'data/bechdel_films.csv' , index = False , encoding = 'utf-8' )

Read back the .csv into a dataframe for later use. Variable types specified as string, otherwise the imdb id's are read as float types. 

In [2]:
bechdel_films = pd.read_csv('data/bechdel_films.csv' , dtype = str , encoding = 'utf-8' ) 

bechdel_films = bechdel_films[ bechdel_films.imdbid.notnull() ].reset_index().drop( ['index' , 'id'] , 1 )

/var/folders/pf/1ppt1rjs6h3dwdsjkpzmyrqc0000gn/T/ipykernel_3202/1331388029.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  bechdel_films = bechdel_films[ bechdel_films.imdbid.notnull() ].reset_index().drop( ['index' , 'id'] , 1 )


### Scraping IMDb for LGBTQ+ tags

We (try to) scrape every film present in the Bechdel dataset.  
The list of tags are saved into a dictionary for later use.  
By having the full list of tags for every film, the data can be used for other projects as well. 

Warning: this code chunk takes hours (roughly 1 second per IMDb Id's and we have over 9000), the results are saved into a .npy file and read back.  

As the code was interrupted as we worked on it, the code below is the "final" product that reads back the already scraped dictionary and checks which Id is left to be scraped. Without the "already scraped base", an empty dictionary should be created first.   
We manually rerun it as long as we reached the final result, but it can be also automated with a while clausule that keeps recycling the id's from the Bechdel dataset until everything is okay.  

In [3]:
try: 
    tag_dict = np.load('data/tag_dictionary.npy', allow_pickle = True ).item()
except FileNotFoundError:
    print("We haven't scraped anything so far, so let's create and empty dictionary")
    tag_dict = {}

In [4]:
len(tag_dict)

9410

In [5]:
to_scrape = [e for e in bechdel_films['imdbid'].to_list() if e not in list(tag_dict.keys()) ]
len(to_scrape)

#it is 0 length, as we scraped everything already. 

0

In [ ]:
j = 0
for i in to_scrape:
    try: 
        res = requests.get("https://www.imdb.com/title/tt" + i + "/keywords?ref_=tt_stry_kw")
        soup = BeautifulSoup(res.text, 'html.parser')
        taglist = soup.find_all('div' , class_="sodatext")
        tag_dict[i] = [ j.text.strip('\n') for j in taglist ]
        j = j + 1 
    except Exception as err: 
        print("For IMDb Id: " + str(imdb_id))
        print(e)

In [ ]:
#np.save('data/tag_dictionary.npy', tag_dict)

In [6]:
tag_dict = np.load('data/tag_dictionary.npy', allow_pickle = True ).item()

### Create LGBTQ+ variable
Let's define our LGBTQ keywords first.    

By checking if a film is LGBTQ+ related, we iterate through the tags/keywords scraped from IMDb and check how many of the keywords (cleaned from punctuation, special characters) is in the the LGBTQ+ keyword list.  

We save it into a dataframe, with following columns: IMDb ID (for joins), number of keywords scraped (it has a meaning if zero), number of LGBTQ+ keyword spotted.  

In [7]:
lgbtq_keywords = [ 'lgbt' , 'lgbtq' ,
                  'gay' , 'gays' , 'gaycharacter' , 'gaykiss' , 'gayinterest' , 
                  'lesbian' , 'lesbians' , 'lesbianinterest' , 
                  'queer' , 
                  'trans' , 'transsexual' , 'transwoman' , 'transman' , 'transgender' , 
                  'bisexual']

In [8]:
df_rows = []
for k , v in tag_dict.items():
    how_gay = sum([ (re.sub( r'\W+', '', vv) in lgbtq_keywords) for vv in v] )
    df_rows.append( [ k , how_gay , len(v) ] ) 

In [9]:
gay_df = pd.DataFrame( df_rows , columns = ['imdbid','lgbtq_keywords_num','keywords_num'])

Time to merge with the Bechdel dataset. 

In [10]:
df = pd.merge( bechdel_films , gay_df , how = 'inner')

In [11]:
#a quick glimpse
df[ df['lgbtq_keywords_num']  > 0 ].head(10)

,title,imdbid,year,rating,lgbtq_keywords_num,keywords_num
52,Dickson Experimental Sound Film,0177707,1894,0,2,16
123,"Eclipse: Courtship of the Sun and Moon, The",0215737,1907,0,2,12
137,A Florida Enchantment,0003973,1914,2,4,28
211,Wings,0018578,1927,1,4,73
213,Downhill [When Boys Leave Home],0017825,1927,1,2,74
232,Pandora's Box (Die Büchse der Pandora),0018737,1929,3,3,122
250,Mädchen in Uniform,0022183,1931,3,1,34
360,"Mandarin Mystery, The",0029207,1936,3,1,51
373,Dracula's Daughter,0027545,1936,3,1,55
409,"Wizard of Oz, The",0032138,1939,3,2,304


We have LGBTQ+ relevant films even among the oldies, and the tags are suprisingly on point. For instance the Die Büchse der Pandora (1929) contains “probably the first explicitly drawn lesbian character” in cinematic history according to film historian Vito Russo. "Good-old-times" in the Weimar era Berlin... Some other films, like the Wizard of Oz has a strong cult in the (American) LGBTQ+ community - see the term "Friends of Dorothy" -, but they don't touch the topic in the film.    
Also woth noting, that these LGBTQ+ oldies all passed the Bechdel-test (except for the first 2, that are silent films so no chance for a conversation between 2 female characters...).  

The tags seem to be user-curated on IMDb, so we shouldn't take everything for granted, but still a good performance.  

### OMDb API
Enriching our dataset, we use the OMDb API (alternatively the IMDb website could be scraped for this information as well).  

Same strategy as with scraping, the already retrieved imdb Id's won't be used. We read the already requested dictionary from a file, if it doesn't exist, an empty dictionary is created. Unused imdb id's are used for the API request only.  

The API requires an API key and comes with a daily 1.000 limit (in reality we actually split the dataset into 2 and run the codes separately, then later merged the result. Here, a one window code is presented). We stocked up with a bunch of API keys by using various e-mail addresses, so we don't run out of them too soon.  
The code recycles the API keys. If it runs into a 'Request limit reached!' error message, it removes the API key from the list and tries until it finds an API key with free limits - see the while clausule in the loop. By only using the unrequested imdb id's (see above), we had the chance to run it on multiple days until we had (almost) everything.  

In [12]:
try: 
    resp_json_dict = np.load('data/resp_json_dict.npy', allow_pickle = True ).item()
except FileNotFoundError:
    print("We haven't requested anything so far, so let's create and empty dictionary")
    resp_json_dict = {}

In [13]:
list_of_imdb_ids = [e for e in df['imdbid'].to_list() if e not in list(resp_json_dict.keys()) ]
len(list_of_imdb_ids)

#it's empty as well, we got everything already

0

In [14]:
API_keys = ['bdcace91' , 
           '5b1dd383' , 
           '18eecc1c' , 
           '59f678e9' , 
           'ea5d7d0c',
           '7ecc3518' , 
           'efbb5d26' , 
           '101156d4' , 
           '78906128' , 
           'c0bebd87' , 
           '7dfca6b4']

In [ ]:
for imdb_id in list_of_imdb_ids:
    try:
        url = 'http://www.omdbapi.com/?i=tt' + imdb_id + '&apikey=' + str(API_keys[0])
        resp = requests.get(url)        
        resp_json = json.loads(resp.text)
        
        #if we happen to run into API limit (1.000 calls per day) and the error is because of the limit,
        #this chunk overwrites the API_keys list, drops the first key we are using until no limit issue
        while (resp_json['Response'] == 'False' and resp_json['Error'] == 'Request limit reached!'):
            API_keys = API_keys[1:]
            url = 'http://www.omdbapi.com/?i=tt' + imdb_id + '&apikey=' + str(API_keys[0])
            resp = requests.get(url)        
            resp_json = json.loads(resp.text)    
        #save the json response into our dictionary    
        resp_json_dict[imdb_id] = resp_json
    
    #if we encounter any type of errors, let's just print it, so the code running for hours doesn't get interrupted.
    #we can return for the missing IMDb Id's later if feeling so
    except Exception as e: 
        print("For IMDb Id: " + str(imdb_id))
        print(e)

Save it (and load back)

In [ ]:
#np.save('data/resp_json_dict.npy' , resp_json_dict)

In [15]:
resp_json_dict = np.load('data/resp_json_dict.npy', allow_pickle = True ).item()

### Clean the dictionary for df transformation
We faced a few issues during the API requests.  
1) The OMDb doesn't contain a few imdb id's (that are on IMDb as we scraped it!)  
2) Sometimes it threw data errors (optionally we can return to the earlier step after we removed these from the dictionary and try the API again, but it won't make any good, error is returned for later tries as well)  
3) Some elements should be a list, but they were returned as a string, separated by comma. See later the data structure exploration.  

First, let's check the errors and remove them. 

In [16]:
id_remove = []
for i in resp_json_dict.keys():
    if len(resp_json_dict[i].keys()) == 2 :
        id_remove.append(i)
        
len(id_remove)

#we have 9 errors, it keeps being 9 no matter how many times we re-run it. 0.1% of the sample so not a big deal. 

9

In [17]:
for i in id_remove:
    del resp_json_dict[i]

**Let's check out the datatypes inside of the dictionary**

In [18]:
key_content_types = {}
for k , v in resp_json_dict.items():
    for kk , vv in v.items():
        key_content_types[kk] = type(vv)

key_content_types

{'Title': str,
 'Year': str,
 'Rated': str,
 'Released': str,
 'Runtime': str,
 'Genre': str,
 'Director': str,
 'Writer': str,
 'Actors': str,
 'Plot': str,
 'Language': str,
 'Country': str,
 'Awards': str,
 'Poster': str,
 'Ratings': list,
 'Metascore': str,
 'imdbRating': str,
 'imdbVotes': str,
 'imdbID': str,
 'Type': str,
 'DVD': str,
 'BoxOffice': str,
 'Production': str,
 'Website': str,
 'Response': str,
 'Season': str,
 'Episode': str,
 'seriesID': str,
 'totalSeasons': str}

Ratings is the only key with list values. The rest is string, even though some are lists in reality, separated by comma.  

#### Focus on useful variables
In the next steps, we are going to focus on variables useful for our analysis. The dictionary is saved, so our data can be enriched anytime with further features.  

We are anyways iterating through the dictionary, so let's also add the missing keys with "N/A" values, so we won't need to raise exceptions or if statements in later codes in order to avoid IndexError.  

In [19]:
keys_to_be_kept = ['Released' , 'Runtime' , 
                   'Director' , 'Actors' , 
                   'Country' , 
                   'imdbRating' , 'imdbVotes' , 
                   'BoxOffice']

for k , v in resp_json_dict.items():
    for i in list(v.keys()):
        if i not in keys_to_be_kept:
            del v[i]
            
    for j in keys_to_be_kept:
        if j not in v.keys():
            v[j] = "N/A"

The following chunk reverses the original dictionary, what are all the (possible) values per all possible keys. This chunk was modified multiple times to check different keys, so we can have a look into it and know what should we do during the cleaning. An iterative and exploratory process, so we leave only an example and won't document everything the way we did.  

The later cleaning steps are based on these explaratory iterations.  

The example: checking if the currencies of the BoxOffice are in a consistent format and not "lists in strings" (starts with $ sign, no deciamals, comma is used as thousand separator or list?).  

What we checked:     
- Directors, actors, countries: really lists stored in string?  
- Running time: everything in minutes?  
- country: co-productions? if yes then ABC order or more important first?   
- currency: $, th separator, decimals  

In [20]:
key_contents = {}

for i in keys_to_be_kept:
    key_contents[i] = []
    
for k , v  in resp_json_dict.items():
    for kk , vv in v.items():
        if kk in list(key_contents.keys()):
            key_contents[kk].append(vv)
        

In [21]:
currencies = []
for i in list(set(key_contents['BoxOffice'])):
    currencies.append(i[0])
set(currencies)

#everything in US dollars, N is the N/A

{'$', 'N'}

**Let's transform those "lists stored in strings" into real lists** to work with. Let's also create a new dictionary by copying the original, so we don't need to go back too much in case of problems. 

In [22]:
fakestrings = ['Director' , 'Actors' , 'Country' ]
clean_dict = resp_json_dict

for j in list(resp_json_dict.keys()):
    
    for i in fakestrings:
        clean_dict[j][i] = resp_json_dict[j][i].split(',')

**Now, let's work with the country data.**  

A large number of films is a co-production. As we investigated earlier, the countries are not in alphabetical order, the first country is the most relevant one.   
- if there are more than one countries, a CoProduction variable is created (1: yes, 0: no, N/A)
- only the first country is kept in the Country variable

In [23]:
for k , v in clean_dict.items():
    if len( v['Country'] ) > 1 :
        v['CoProduction'] = '1'
    elif v['Country'][0] == "N/A" :
        v['CoProduction'] = "N/A"
    else: 
        v['CoProduction'] = "0"
    v['Country'] = v['Country'][0]

### Gender recognition ML
In order to make sense of the Directors and Actors data, we run a gender recognition ML tool on the dataset.  

The ML tool takes the first name (fails on full name, so we take the name before the first white space) as an argument and returns wether it is (more likely) male or female, androgynous (can't decide), or unknown in their dictionary.  

The director's gender is important for our research question.  
The Actor's gender is practically reflected in the Bechdel Test, we rather use it to control for gay-male romance films that naturally fail on the Bechdel-test by design.  

We won't care about N/A here, if the ML fails to recognise the name, then unknown is anyways going to be treated the same way.  

In [24]:
d = gen.Detector()

for k , v in clean_dict.items():
    v['DirectorGender'] = [ d.get_gender( i.split()[0] ) for i in v['Director'] ]
    v['ActorGender'] = [ d.get_gender( i.split()[0] ) for i in v['Actors'] ]

Summarise the results into single variables.  

Director gender: if at least one of the directors is (more likely) female, it takes the value 'female'. Otherwise, if at least one is (more likely) male, then 'male', if everyone is unknown or ambigous angrogyn, then 'N/A'.

Actors' gender: if ALL (more likely) male, then 'male'. Otherwise if at least one is identified as (more) female, then 'female', for the rest 'N/A'.

In [25]:
for k , v in clean_dict.items():
    dir_male = sum( [ ( i in ['male' , 'more_male']) for i in v['DirectorGender'] ] )
    dir_female = sum( [ ( i in ['female' , 'more_female']) for i in v['DirectorGender'] ] )
    
    if dir_female > 0:
        v['DirectorGender'] = 'female'
    elif dir_male > 0:
        v['DirectorGender'] = 'male'
    else: 
        v['DirectorGender'] = 'N/A'
        
    actor_male = sum( [ ( j in ['male' , 'more_male']) for j in v['ActorGender'] ] )
    actor_female = sum( [ ( j in ['female' , 'more_female']) for j in v['ActorGender'] ] )
    
    if actor_male == len( v['ActorGender'] ):
        v['ActorGender'] = 'male'
    elif actor_female > 0:
        v['ActorGender'] = 'female'
    else: 
        v['ActorGender'] = 'N/A'

Now, let's remove the lists, so we can simply transform the dictionary into a df:

In [26]:
for k , v in clean_dict.items():
    del v['Actors']
    del v['Director']

### Cleaning in pandas

We used the dictionaries enough, let's transform the cleaned dictionary into a pandas df, merge with the Bechdel/LGBTQ+ dataset and execute the remaining cleaning steps there.  

In [27]:
imdb_api_df = pd.DataFrame.from_dict( clean_dict , orient = 'index' )
imdb_api_df.index.names = ['imdbid'] 
imdb_api_df.reset_index( inplace = True )
imdb_api_df.head()

,imdbid,Released,Runtime,Country,imdbRating,imdbVotes,BoxOffice,CoProduction,DirectorGender,ActorGender
0,0775529,01 Feb 2008,113 min,United States,7.1,"37,122","$6,623,082",0,female,female
1,0459102,18 Oct 2007,141 min,Austria,7.0,"5,807",N/A,1,male,female
2,0452702,20 Apr 2007,85 min,United States,6.2,"99,055","$19,363,565",0,N/A,female
3,0480249,14 Dec 2007,101 min,United States,7.2,"732,538","$256,393,010",0,male,female
4,0448134,27 Jul 2007,107 min,United Kingdom,7.2,"245,858","$3,675,753",1,male,female


In [28]:
final_df = pd.merge( df , imdb_api_df ,
                     how = 'inner' )
final_df.tail()

,title,imdbid,year,rating,lgbtq_keywords_num,keywords_num,Released,Runtime,Country,imdbRating,imdbVotes,BoxOffice,CoProduction,DirectorGender,ActorGender
9400,Being The Ricardos,4995540,2021,3,0,62,21 Dec 2021,131 min,United States,6.6,"35,406",N/A,0,male,female
9401,Boiling Point,11127680,2021,3,0,45,07 Jan 2022,92 min,United Kingdom,7.5,"7,617",N/A,0,male,female
9402,"Power of the Dog, The",10293406,2021,0,3,174,01 Dec 2021,126 min,United Kingdom,6.9,"118,298",N/A,1,female,female
9403,Mass,11389748,2021,3,0,10,15 Oct 2021,111 min,United States,7.7,"9,486","$145,174",0,female,female
9404,"355, The",8356942,2022,3,0,71,07 Jan 2022,122 min,United States,5.2,"17,644","$14,570,455",1,male,female


In [29]:
final_df.dtypes

title                 object
imdbid                object
year                  object
rating                object
lgbtq_keywords_num     int64
keywords_num           int64
Released              object
Runtime               object
Country               object
imdbRating            object
imdbVotes             object
BoxOffice             object
CoProduction          object
DirectorGender        object
ActorGender           object
dtype: object

**Let's clean the Runtime and BoxOffice variable and imdbVotes** from all non-numerical characters, so the commas (th separators), dollar sings, ' min' parts are removed. We checked that the formats are consistent, so no need to worry about various thousand separators, etc.  

In [30]:
final_df['Runtime'] = final_df['Runtime'].apply( lambda x: re.sub( "[^0-9]", "", x ) )
final_df['BoxOffice'] = final_df['BoxOffice'].apply(lambda x: re.sub( "[^0-9]", "", x ) )
final_df['imdbVotes'] = final_df['imdbVotes'].apply(lambda x: re.sub( "[^0-9]", "", x ) )

In [31]:
final_df.head()

,title,imdbid,year,rating,lgbtq_keywords_num,keywords_num,Released,Runtime,Country,imdbRating,imdbVotes,BoxOffice,CoProduction,DirectorGender,ActorGender
0,Passage de Venus,3155794,1874,0,0,9,09 Dec 1874,1,France,6.9,1586,,0,N/A,N/A
1,La Rosace Magique,14495706,1877,0,0,1,03 Jan 1877,1,France,N/A,86,,0,N/A,N/A
2,Sallie Gardner at a Gallop,2221420,1878,0,0,7,15 Jun 1878,1,United States,7.4,2849,,0,N/A,female
3,Le singe musicien,12592084,1878,0,0,8,07 May 1878,1,France,6.0,181,,0,N/A,N/A
4,Athlete Swinging a Pick,7816420,1881,0,0,8,01 Jan 1881,1,United Kingdom,5.2,397,,0,N/A,N/A


It is enough information if the given film has at least one LGBTQ-related tag. The total numer of tags isn't relevant. 

In [32]:
final_df['IsItGay'] = final_df['lgbtq_keywords_num'].apply( lambda x: 1 if x > 0 else 0 )

In [33]:
final_df = final_df.drop( ['lgbtq_keywords_num' , 'keywords_num'] , 1 )

/var/folders/pf/1ppt1rjs6h3dwdsjkpzmyrqc0000gn/T/ipykernel_3202/289946624.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  final_df = final_df.drop( ['lgbtq_keywords_num' , 'keywords_num'] , 1 )


Finally, let's replace the empty cells and 'N/A' with real missing values

In [34]:
final_df.replace('N/A', np.NaN , inplace = True )
final_df.replace('', np.NaN , inplace = True )

Save our cleaned and final df into a csv file. 

In [35]:
#final_df.to_csv( 'data/clean_df.csv' , index = False , encoding = 'utf-8' )

### DataViz
First let's read back the data and make sure the columns have the approriate datatypes

In [36]:
clean_df = pd.read_csv('data/clean_df.csv' , dtype = str , encoding = 'utf-8' ) 

# Nadine, 

here is the point where you should read back this df. The data is cleaned, but everything is read back as string so the automatic data type recognition doesn't screw it up.  

Not sure how you want them exactly, here are the lists for how I think it would be great + the date and numerics already transformed.  
Be careful, the missing values are really missing values and lambda is tricky with NaN. It's an analyst decision how we want to deal with NaN, like fill it with other values or drop the observations in some cases.   

Also, I made sure that the saving commands are commented, so you won't overwrite anything. The files are clean and neat, all of them, and more importatnly, the file names are consistent in this notebook. So it's way easier to pull everything for GitHub and let the relative references do their job and forget about every kind of earlier file versions. 

In [38]:
numeric_vars = [ 'year' , 'Runtime' , 'imdbVotes' , 'BoxOffice' , 'imdbVotes' ]
date_vars = [ 'Released' ]
binary_vars = [ 'CoProduction' , 'IsItGay' ]
categorical_vars = [ 'rating' , 'DirectorGender' , 'ActorGender']

for i in numeric_vars:
    clean_df[i] = pd.to_numeric(clean_df[i])

clean_df['Released'] = pd.to_datetime(clean_df['Released'])

In [39]:
clean_df.dtypes

title                     object
imdbid                    object
year                       int64
rating                    object
Released          datetime64[ns]
Runtime                  float64
Country                   object
imdbRating                object
imdbVotes                float64
BoxOffice                float64
CoProduction              object
DirectorGender            object
ActorGender               object
IsItGay                   object
dtype: object